In [1]:

from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig
from tqdm.autonotebook import tqdm as notebook_tqdm
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

/workspace/vinhnq/miniconda3/envs/RAG/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
MILVUS_HOST = "localhost"
MILVUS_PORT = "19530"
from langchain_community.embeddings import HuggingFaceEmbeddings
# Load model embedding
model_name = "Alibaba-NLP/gte-large-en-v1.5"
model_kwargs = {'device': 'cuda', 'trust_remote_code': True}
encode_kwargs = {'normalize_embeddings': False}
cache_folder = "/workspace/vinhnq/cache_weights"
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
    cache_folder=cache_folder
)

/workspace/vinhnq/miniconda3/envs/RAG/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/workspace/vinhnq/miniconda3/envs/RAG/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
from langchain.vectorstores import Milvus
# Kết nối với database đã tạo sẵn
NAME="Vinh"
vector_db = Milvus(
    embeddings,
    connection_args={"host": MILVUS_HOST, "port": MILVUS_PORT},
    collection_name=NAME,
)
import transformers
import torch
from transformers import BitsAndBytesConfig
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from huggingface_hub import login
from langchain.llms import HuggingFacePipeline
#login(token = "Your token")
os.environ["CUDA_VISIBLE_DEVICES"]="2"

# Gọi model llama
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    return_full_text=True,
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
    temperature=0.7,
    max_new_tokens=512,
    repetition_penalty=1,
    num_return_sequences=1,
    )
llm = HuggingFacePipeline(pipeline=pipeline)


/workspace/vinhnq/miniconda3/envs/RAG/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `Milvus` was deprecated in LangChain 0.2.0 and will be removed in 0.3.0. An updated version of the class exists in the langchain-milvus package and should be used instead. To use it run `pip install -U langchain-milvus` and import as `from langchain_milvus import MilvusVectorStore`.
  warn_deprecated(


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/vinhnq/.cache/huggingface/token
Login successful


Loading checkpoint shards: 100%|██████████| 4/4 [00:04<00:00,  1.03s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/workspace/vinhnq/miniconda3/envs/RAG/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(


In [6]:
import os
import transformers
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from huggingface_hub import login
from langchain.llms import HuggingFacePipeline
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
first_time = True
# Khởi tạo retriever
retriever = vector_db.as_retriever(search_kwargs={"k": 5})
context = retriever
# Contextualize question
contextualize_q_system_prompt = """Please extract relevant information from previous question and answer pairs between you and the user to support your response to the next question from the computer science expert."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

# Prompt để trả lời câu hỏi
qa_prompt_template = """
Your name is Vinbot, a chatbot designed to assist experts in the field of computer science using the Retrieval-Augmented Generation (RAG) technique. Your task is to provide accurate, detailed, and insightful answers on topics related to computer science. Here are specific requirements:

You must consider the answer based on the context below, which is information extracted from relevant documents, but remember not to explicitly state that you are answering based on context. Below is the retrieved context and the user's question:

Context:
{context}

Question:
{input}
1. Ensure that your answers are based on the latest and most accurate information available.
2. If the question exceeds your current knowledge, guide users to relevant reference materials or suggest keywords for further research, and if you do not know, answer that you do not know.
3. For technical terms, provide answers with abbreviations, for example, Deep Learning (DL), Machine Learning (ML).
4. Structure your answer like ChatGPT:
- Emphasize main points with headings or numbered sections.
- Use bullet points or numbering for important lists or steps.
- Bold important notes or warnings with (*) markers.
Give detailed answers to questions
"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_prompt_template),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
'''
# Prompt để trả lời câu hỏi lần đầu tiên
qa_prompt_template_first_time = """
Your name is Vinbot, a chatbot designed to assist experts in the field of computer science using the Retrieval-Augmented Generation (RAG) technique. Your task is to provide accurate, detailed, and insightful answers on topics related to computer science. Here are specific requirements:

You must consider the answer based on the context below, which is information extracted from relevant documents, but remember not to explicitly state that you are answering based on context. Below is the retrieved context and the user's question:

Context:
{context}

Question:
{input}
1. Ensure that your answers are based on the latest and most accurate information available.
2. If the question exceeds your current knowledge, guide users to relevant reference materials or suggest keywords for further research, and if you do not know, answer that you do not know.
3. For technical terms, provide answers with abbreviations, for example, Deep Learning (DL), Machine Learning (ML).
4. Structure your answer like ChatGPT:
- Emphasize main points with headings or numbered sections.
- Use bullet points or numbering for important lists or steps.
- Bold important notes or warnings with (*) markers.
Give detailed answers to questions
"""

# Prompt để trả lời câu hỏi từ lần thứ hai trở đi
qa_prompt_template_repeat = """
Question:
{input}
"""

# Biến để theo dõi số lần gọi hàm
call_count = 1

def get_qa_prompt_template():
    global call_count
    call_count += 1
    if call_count == 1:
        return qa_prompt_template_first_time
    else:
        return qa_prompt_template_repeat

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", get_qa_prompt_template()),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
'''

question_answer_chain = create_stuff_documents_chain(llm.bind(skip_prompt = True), qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


In [7]:
# Statefully manage chat history
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

# Function to invoke the chain with session_id
def invoke_chain(session_id, user_input):
    response = conversational_rag_chain.invoke(
        {"input": user_input},
        config={"configurable": {"session_id": session_id}}
    )["answer"]
    return response

In [8]:
def store_to_string(store):
    store_str = ""
    for session_id, messages in store.items():
        store_str += f"Session ID: {session_id}\n"
        for message in messages:
            if isinstance(message, str) and "AIMessage(content=" in message:
                # Xuống dòng nếu là tin nhắn từ AI
                store_str += f"{message}\n\n"
            else:
                store_str += f"{message}\n"
        store_str += "\n"
    return store_str

# Hàm lưu lịch sử hội thoại vào file text
def save_store_to_file(file_path: str):
    store_str = store_to_string(store)
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(store_str)

# Hàm in nội dung của store
def print_store():
    store_str = store_to_string(store)
    print(store_str)
# Lưu store vào file text
save_store_to_file("store_history.txt")

# In nội dung của store
print_store()

In [ ]:
# Test the chatbot with session history
response1 = invoke_chain("session_0", "what is machine learning in the field of AI ?")
print(response1)

Parent run ed6abf24-b795-46bf-b0e1-55853d1261a6 not found for run d3adacb5-8581-4bdb-b299-366258abddc4. Treating as a root run.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


System: 
Your name is Vinbot, a chatbot designed to assist experts in the field of computer science using the Retrieval-Augmented Generation (RAG) technique. Your task is to provide accurate, detailed, and insightful answers on topics related to computer science. Here are specific requirements:

You must consider the answer based on the context below, which is information extracted from relevant documents, but remember not to explicitly state that you are answering based on context. Below is the retrieved context and the user's question:

Context:
9:2 A.Esmaeili et al.
1 INTRODUCTION
Machine Learning (ML) is a particularly prevalent branch of Artificial Intelligence that is be-
comingincreasinglyrelevanttoreal-lifeapplications,including,butnotlimitedto,economics[ 6],
education[ 24],agriculture[ 44],drugdiscovery[ 68],andmedicine[ 55].Sucharapidgrowth,fueled
by either the emergence of new data/applications or the challenges in improving the generality
of the solutions, demands straightf

In [ ]:
print(store)

{'session_0': InMemoryChatMessageHistory(messages=[HumanMessage(content='what is machine learning in the field of AI ?'), AIMessage(content='System: \nYour name is Vinbot, a chatbot designed to assist experts in the field of computer science using the Retrieval-Augmented Generation (RAG) technique. Your task is to provide accurate, detailed, and insightful answers on topics related to computer science. Here are specific requirements:\n\nYou must consider the answer based on the context below, which is information extracted from relevant documents, but remember not to explicitly state that you are answering based on context. Below is the retrieved context and the user\'s question:\n\nContext:\n9:2 A.Esmaeili et al.\n1 INTRODUCTION\nMachine Learning (ML) is a particularly prevalent branch of Artificial Intelligence that is be-\ncomingincreasinglyrelevanttoreal-lifeapplications,including,butnotlimitedto,economics[ 6],\neducation[ 24],agriculture[ 44],drugdiscovery[ 68],andmedicine[ 55].Su

In [ ]:
response2 = invoke_chain("session_1", "robot in the World")
print(response2)

Parent run 34053f11-3720-409e-aadc-1fd2eeb28ba9 not found for run 5029b74d-00cf-4a1f-bd8a-1c1476db9ac2. Treating as a root run.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


System: 
Your name is Vinbot, a chatbot designed to assist experts in the field of computer science using the Retrieval-Augmented Generation (RAG) technique. Your task is to provide accurate, detailed, and insightful answers on topics related to computer science. Here are specific requirements:

You must consider the answer based on the context below, which is information extracted from relevant documents, but remember not to explicitly state that you are answering based on context. Below is the retrieved context and the user's question:

Context:
referees, and the editors for their constructive comments andvaluable suggestions, which have helped improve the quality
and presentation of the paper.
R
EFERENCES
[1] Crazyﬂie 2.1—Bitcraze Store . Accessed: Jul. 22, 2021. [Online].
Available: https://www.bitcraze.io/2019/02/the-crazyﬂie-2-1-is-here/
[2] RoboMaster EP—DJI . Accessed: Jul. 22, 2021. [Online]. Available:
https://www.dji.com/robomaster-ep
[3] Boston Dynamics Launches Commercial 

In [ ]:
response2 = invoke_chain("session_1", "AI in the World")
print(response2)

Parent run 67fbb4a2-f393-41b8-927e-178a14e96d3d not found for run 1d010819-669c-469f-9294-b5e8d49ca07c. Treating as a root run.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


System: 
Your name is Vinbot, a chatbot designed to assist experts in the field of computer science using the Retrieval-Augmented Generation (RAG) technique. Your task is to provide accurate, detailed, and insightful answers on topics related to computer science. Here are specific requirements:

You must consider the answer based on the context below, which is information extracted from relevant documents, but remember not to explicitly state that you are answering based on context. Below is the retrieved context and the user's question:

Context:
referees, and the editors for their constructive comments andvaluable suggestions, which have helped improve the quality
and presentation of the paper.
R
EFERENCES
[1] Crazyﬂie 2.1—Bitcraze Store . Accessed: Jul. 22, 2021. [Online].
Available: https://www.bitcraze.io/2019/02/the-crazyﬂie-2-1-is-here/
[2] RoboMaster EP—DJI . Accessed: Jul. 22, 2021. [Online]. Available:
https://www.dji.com/robomaster-ep
[3] Boston Dynamics Launches Commercial 

In [9]:
import gradio as gr

# Biến lưu trữ mô hình và các tham số hiện tại
current_pipeline = llm.pipeline
current_temperature = 0.4
current_max_new_tokens = 1024
current_repetition_penalty = 1

# Hàm khởi tạo pipeline với các tham số điều chỉnh
def create_pipeline(temperature, max_new_tokens, repetition_penalty):
    return transformers.pipeline(
        "text-generation",
        return_full_text=True,
        model=model_id,
        model_kwargs={"torch_dtype": torch.bfloat16},
        device_map="auto",
        temperature=temperature,
        max_new_tokens=max_new_tokens,
        repetition_penalty=repetition_penalty,
        num_return_sequences=1,
    )

# Hàm xử lý đầu vào từ người dùng và trả về kết quả từ chatbot
def chat_with_bot(user_input, history, temperature, max_new_tokens, repetition_penalty, session_input):
    global current_pipeline, current_temperature, current_max_new_tokens, current_repetition_penalty,session
    session = session_input
    # Kiểm tra xem các tham số có thay đổi không
    if (current_pipeline is None or 
        temperature != current_temperature or 
        max_new_tokens != current_max_new_tokens or 
        repetition_penalty != current_repetition_penalty):

        # Tạo pipeline với các tham số từ giao diện
        current_pipeline = create_pipeline(temperature, max_new_tokens, repetition_penalty)
        current_temperature = temperature
        current_max_new_tokens = max_new_tokens
        current_repetition_penalty = repetition_penalty
    
    llm = HuggingFacePipeline(pipeline=current_pipeline)

    # Cấu hình lại rag_chain với các thành phần mới

    # Gọi rag_chain để lấy kết quả
    output = invoke_chain(session, user_input)
    # Cập nhật lịch sử chat
    history.append((user_input, output))
    save_store_to_file("store_history.txt")
    return "", history

# CSS tùy chỉnh để tạo giao diện giống ChatGPT
css = """
.gradio-container {
    font-family: 'Arial', sans-serif;
    background: #f5f5f5;
    color: #333;
    padding: 20px;
    height: 100vh;
}

#chatbox {
    max-width: 90%;
    height: 100%;
    margin: auto;
    border: 1px solid #ccc;
    border-radius: 10px;
    padding: 10px;
    background: white;
    box-shadow: 0 4px 8px rgba(0,0,0,0.1);
    display: flex;
    flex-direction: column;
}
#header {
    display: flex;
    position: relative
    justify-content: center; /* Căn giữa các thành phần theo chiều ngang */
    align-items: center; /* Căn giữa các thành phần theo chiều dọc */
    text-align: center;
    padding: 10px;
    background-color: #4CAF50;
    color: white;
    border-radius: 10px 10px 0 0;
}

.header-content {
    display: flex;
    flex-direction: column;
    align-items: center;
    width: 100%;
}
#footer {
    text-align: center;
    padding: 10px;
    color: #888;
    margin-top: 20px;
    font-size: 0.9em;
}

#user_input_row {
    display: flex;
    width: 100%;
    padding: 5px;
    border-top: 1px solid #ccc;
    background-color: #f9f9f9;
    justify-content: space-between;
}

#user_input {
    flex: 1; /* Ô nhập liệu sẽ chiếm hết không gian còn lại */
}

#submit_button {
    background-color: green;
    flex: 0.1;
    color: black;
    width: auto; /* Đặt lại chiều rộng tự động để phù hợp với nội dung */
    border: none;
    padding: 8px 16px; /* Điều chỉnh padding để nút nhìn đẹp hơn */
    border-radius: 5px;
    cursor: pointer;
    transition: background-color 0.3s ease;
}

#submit_button:hover {
    background-color: #45A049;
}

#session_input {
    flex: 0.2;
    border-radius: 5px;
    border: 1px solid #ccc;
    padding: 5px;
    max-width: 100px;
    top: 20px;
    right: 20px;
    position: absolute;

}

"""
    
# Tạo giao diện với Gradio
with gr.Blocks(css=css) as iface:
    gr.HTML(
        """
        <div id="header">
            <div class="header-content">
                <h1 class="centered-h1">Chatbot Khoa học máy tính</h1>
                <p class="centered-p">Được hỗ trợ bởi mô hình Meta-Llama. Đặt câu hỏi của bạn về khoa học máy tính và nhận câu trả lời chi tiết!</p>
            </div>
            <input type="text" id="session_input" placeholder="Nhập session">
        </div>
        """
    )
    
    with gr.Column(elem_id="chatbox"):
        chatbot = gr.Chatbot()
        state = gr.State([])
        
        with gr.Row(elem_id="user_input_row"):  # Sử dụng elem_id để áp dụng CSS riêng
            user_input = gr.Textbox(
                show_label=False, 
                placeholder="Nhập câu hỏi của bạn...",
                elem_id="user_input"
            )
            
            submit_button = gr.Button("Gửi câu hỏi", elem_id="submit_button")

        with gr.Row():
            temperature = gr.Slider(0.1, 1.0, value=current_temperature, step=0.1, label="Temperature")
            max_new_tokens = gr.Slider(10, 1024, value=current_max_new_tokens, step=10, label="Max New Tokens")
            repetition_penalty = gr.Slider(1.0, 2.0, value=current_repetition_penalty, step=0.1, label="Repetition Penalty")
        user_input.submit(chat_with_bot, [user_input, state, temperature, max_new_tokens, repetition_penalty, gr.Textbox(elem_id="session_input",visible=False)], [user_input, chatbot])
        submit_button.click(chat_with_bot, [user_input, state, temperature, max_new_tokens, repetition_penalty, gr.Textbox(elem_id="session_input",visible=False)], [user_input, chatbot])
        


    gr.HTML(
        """
        <div id="footer">
            <p>&copy; 2024 Chatbot Khoa học máy tính. Được phát triển bởi Nhóm Khoa học Máy tính.</p>
        </div>
        """
    )
# Khởi chạy giao diện
iface.launch(share=True)


Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://fc858a79c9f6350ae9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
